In [16]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

In [17]:
#читаем данные
df = pd.read_csv('adult.csv')

In [20]:
#смотрим информацию датасета
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45222 entries, 0 to 48841
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             45222 non-null  int64 
 1   workclass       45222 non-null  object
 2   fnlwgt          45222 non-null  int64 
 3   education       45222 non-null  object
 4   education-num   45222 non-null  int64 
 5   marital-status  45222 non-null  object
 6   occupation      45222 non-null  object
 7   relationship    45222 non-null  object
 8   race            45222 non-null  object
 9   sex             45222 non-null  object
 10  capitalgain     45222 non-null  int64 
 11  capitalloss     45222 non-null  int64 
 12  hoursperweek    45222 non-null  int64 
 13  native-country  45222 non-null  object
 14  class           45222 non-null  object
dtypes: int64(6), object(9)
memory usage: 5.5+ MB


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capitalgain,capitalloss,hoursperweek,native-country,class
0,2,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,1,0,2,United-States,<=50K
1,3,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,0,United-States,<=50K
2,2,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,2,United-States,<=50K
3,3,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,2,United-States,<=50K
4,1,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,2,Cuba,<=50K


In [19]:
#удалим строки, где есть пустые значения
df.dropna(inplace = True)
#найдем категориальные переменные из переменных типа int64
print(df['class'].unique(), '\n', df['age'].unique(), '\n', df['capitalgain'].unique(), '\n', \
      df['capitalloss'].unique(), '\n', df['hoursperweek'].unique())
#предполагаю, что это все именно категории

['<=50K' '>50K'] 
 [2 3 1 0 4] 
 [1 0 4 2 3] 
 [0 3 1 2 4] 
 [2 0 3 4 1]


In [6]:
#получим массив целевой переменной, закодируем классы
Y = [0 if x == '<=50K' else 1 for x in df['class']]

In [7]:
#закодируем категориальные переменные, уберем проблему мультиколинеарности (?)
df['age'] = df['age'].astype('category')
df['capitalgain'] = df['capitalgain'].astype('category')
df['capitalloss'] = df['capitalloss'].astype('category')
df['hoursperweek'] = df['hoursperweek'].astype('category')

X_dumm = pd.get_dummies(df.drop(['class', 'fnlwgt'], axis = 1), drop_first = True)

#стандартизируем переменные
X_dumm_scalled = StandardScaler().fit_transform(X_dumm)

In [8]:
#получим тренировочный и тестовый сэмплы
X_train, X_test, Y_train, Y_test = train_test_split(X_dumm_scalled, Y, test_size = 0.3, \
                                                    random_state = 1, stratify = Y)

In [9]:
#------------- Logistic Regression -------------------------

In [10]:
lrc = LogisticRegression(random_state = 42).fit(X_train, Y_train)
lrc.score(X_train, Y_train), lrc.predict_proba(X_train)

(0.8526615068709524,
 array([[0.95048217, 0.04951783],
        [0.6790194 , 0.3209806 ],
        [0.63642399, 0.36357601],
        ...,
        [0.97354797, 0.02645203],
        [0.12135552, 0.87864448],
        [0.86125818, 0.13874182]]))

In [11]:
lrc.score(X_test, Y_test), lrc.predict_proba(X_test)

(0.8552369720645684,
 array([[8.86201620e-01, 1.13798380e-01],
        [9.93265003e-01, 6.73499743e-03],
        [5.97142720e-01, 4.02857280e-01],
        ...,
        [3.16926286e-04, 9.99683074e-01],
        [5.18297851e-01, 4.81702149e-01],
        [9.73605370e-01, 2.63946299e-02]]))

In [12]:
#------------- SVM -------------------------

In [13]:
%%time
svc = SVC(random_state = 42, gamma = 'auto').fit(X_train, Y_train)

CPU times: user 1min 50s, sys: 1.81 s, total: 1min 51s
Wall time: 1min 55s


In [14]:
%%time
svc.score(X_train, Y_train)

CPU times: user 2min 26s, sys: 1.7 s, total: 2min 27s
Wall time: 2min 32s


0.8624229979466119

In [15]:
%%time
svc.score(X_test, Y_test)

CPU times: user 1min 4s, sys: 817 ms, total: 1min 4s
Wall time: 1min 7s


0.8511093093535785

In [ ]:
#--------------- Вывод -----------------------
# Logistic Regression и SVM в лайт версии (без подбора гиперпараметров) показали одинаковый результат